In [32]:
import numpy as np
import xarray as xr
import pandas as pd # Required for time series handling
import os # For path manipulation

In [33]:
def parse_grads_ctl(ctl_file_path):
    """
    Parses a GrADS .ctl file to extract metadata for dimensions and variables.

    Args:
        ctl_file_path (str): The path to the GrADS .ctl file.

    Returns:
        dict: A dictionary containing parsed metadata like 'dset', 'undef',
              'xdef', 'ydef', 'tdef', 'zdef', 'num_vars', and 'variables'.
    """
    metadata = {}
    original_lines = [] # To keep original line content for accurate splitting/parsing
    with open(ctl_file_path, 'r') as f:
        original_lines = f.readlines()

    # Create a lowercased, stripped version of lines for keyword matching
    lines_processed = [line.strip().lower() for line in original_lines]
    
    vars_start_index = -1 # To mark the beginning of the VARS section

    for i, line_content in enumerate(lines_processed):
        if line_content.startswith("dset"):
            # Use original_lines[i] to get the exact string for splitting
            metadata['dset'] = original_lines[i].strip().split()[1].strip('^')
        elif line_content.startswith("undef"):
            metadata['undef'] = float(original_lines[i].strip().split()[1])
        elif line_content.startswith("title"):
            # Capture the full title string after 'title' keyword
            metadata['title'] = original_lines[i].strip().split(' ', 1)[1]
        elif line_content.startswith("options"):
            metadata['byte_order'] = original_lines[i].strip().split()[1]
        elif line_content.startswith("xdef"):
            parts = original_lines[i].strip().split()
            metadata['xdef'] = {'count': int(parts[1]), 'type': parts[2], 'start': float(parts[3]), 'increment': float(parts[4])}
        elif line_content.startswith("ydef"):
            parts = original_lines[i].strip().split()
            metadata['ydef'] = {'count': int(parts[1]), 'type': parts[2], 'start': float(parts[3]), 'increment': float(parts[4])}
        elif line_content.startswith("tdef"):
            parts = original_lines[i].strip().split()
            metadata['tdef'] = {'count': int(parts[1]), 'type': parts[2], 'start_str': parts[3], 'increment_str': parts[4]}
        elif line_content.startswith("zdef"):
            parts = original_lines[i].strip().split()
            metadata['zdef'] = {'count': int(parts[1]), 'type': parts[2], 'start': float(parts[3]), 'increment': float(parts[4])}
        elif line_content.startswith("vars"):
            metadata['num_vars'] = int(line_content.split()[1])
            metadata['variables'] = []
            vars_start_index = i # Store the index of the 'vars' line

    # After iterating through all lines, parse variables using the stored index
    if vars_start_index != -1:
        for j in range(vars_start_index + 1, len(original_lines)):
            sub_line = original_lines[j].strip()
            if sub_line.lower() == "endvars":
                break # Stop when 'endvars' is encountered
            
            parts = sub_line.split()
            if parts: # Ensure the line is not empty
                var_name = parts[0]
                description = ''
                # Attempt to find description after '**'
                if '**' in parts:
                    desc_start_idx = parts.index('**') + 1
                    description = " ".join(parts[desc_start_idx:])
                else:
                    # Heuristic for cases without '**': take remaining parts as description
                    # This might need adjustment based on specific CTL file formats
                    if len(parts) > 2: # e.g., 'sk1 0 59,1,0'
                        description = " ".join(parts[2:]) 
                
                metadata['variables'].append({'name': var_name, 'description': description.strip()})
    
    return metadata

In [34]:
def convert_grads_to_netcdf(ctl_file_path, output_nc_file):
    """
    Converts a GrADS binary data file (.dat) associated with a .ctl file
    into a NetCDF (.nc) file using xarray.

    Args:
        ctl_file_path (str): The path to the GrADS .ctl control file.
        output_nc_file (str): The desired path for the output NetCDF file.
    """
    metadata = parse_grads_ctl(ctl_file_path)

    # Construct the data file path (assumes .dat is in the same directory)
    data_file_path = os.path.join(os.path.dirname(ctl_file_path), metadata['dset'])
    
    # Check if the data file specified in dset uses '^' relative path.
    # If dset starts with '^', it means it's relative to the ctl file location.
    # Otherwise, it could be an absolute path or relative to the CWD.
    # We already handled '^' by stripping it during metadata parsing.
    # Ensure data_file_path is correct if 'dset' had a full path.
    if not os.path.isabs(metadata['dset']) and not metadata['dset'].startswith('^'):
        # If it's a simple filename without '^' or absolute path, assume it's in the same dir
        data_file_path = os.path.join(os.path.dirname(ctl_file_path), metadata['dset'])
    else:
        # If dset was an absolute path or already handled for relative, use it as is
        data_file_path = metadata['dset'].strip('^')


    # 1. Infer spatial dimensions and coordinates
    x_coords = np.arange(metadata['xdef']['start'],
                         metadata['xdef']['start'] + metadata['xdef']['count'] * metadata['xdef']['increment'],
                         metadata['xdef']['increment'])
    y_coords = np.arange(metadata['ydef']['start'],
                         metadata['ydef']['start'] + metadata['ydef']['count'] * metadata['ydef']['increment'],
                         metadata['ydef']['increment'])
    z_coords = np.arange(metadata['zdef']['start'],
                         metadata['zdef']['start'] + metadata['zdef']['count'] * metadata['zdef']['increment'],
                         metadata['zdef']['increment'])

    # 2. Time parsing: Convert GrADS time format to pandas DatetimeIndex
    # Handle GrADS specific time formats, e.g., '00Z01jan1991 1yr'
    try:
        # GrADS date format like '00Z01jan1991' -> '%HZ%d%b%Y'
        start_date = pd.to_datetime(metadata['tdef']['start_str'], format='%HZ%d%b%Y')
    except ValueError:
        # Fallback for other date formats if needed
        print(f"Warning: Could not parse start date '{metadata['tdef']['start_str']}' with '%HZ%d%b%Y'. Attempting generic parse.")
        start_date = pd.to_datetime(metadata['tdef']['start_str']) 

    # Map GrADS frequency strings to pandas frequency strings
    # Add more mappings as necessary based on your .ctl files
    freq_map = {'1yr': 'YS', '1mo': 'MS', '1dy': 'D', '1hr': 'H', 
                '1mn': 'min', '1sc': 'S'} # YS for year start, MS for month start
    time_freq = metadata['tdef']['increment_str'].lower()
    if time_freq in freq_map:
        time_freq = freq_map[time_freq]
    else:
        print(f"Warning: Could not directly map GrADS time increment '{metadata['tdef']['increment_str']}' to pandas frequency. Trying simple replacement.")
        # Attempt simple replacements for common patterns
        if 'yr' in time_freq: time_freq = time_freq.replace('yr', 'Y')
        elif 'mo' in time_freq: time_freq = time_freq.replace('mo', 'M')
        elif 'dy' in time_freq: time_freq = time_freq.replace('dy', 'D')
        elif 'hr' in time_freq: time_freq = time_freq.replace('hr', 'H')
        elif 'mn' in time_freq: time_freq = time_freq.replace('mn', 'min')
        elif 'sc' in time_freq: time_freq = time_freq.replace('sc', 'S')
        else:
            print(f"Error: Unable to determine pandas frequency from '{metadata['tdef']['increment_str']}'. Defaulting to 'D'. This may cause incorrect time coordinates.")
            time_freq = 'D' # Last resort fallback

    time_coords = pd.date_range(start=start_date, periods=metadata['tdef']['count'], freq=time_freq)

    # 3. Determine data type and byte order
    # Use 'float32' for the base dtype, as GrADS binary data is typically 4-byte floats
    base_dtype = np.float32 
    
    # Determine endianness: '<' for little-endian, '>' for big-endian
    endian = '<' if metadata.get('byte_order', 'little_endian').lower() == 'little_endian' else '>'
    
    # Construct the full dtype string with explicit byte order for robustness
    # 'f' for float, then itemsize (e.g., 'f4' for float32)
    data_dtype = np.dtype(f"{endian}f{base_dtype().itemsize}")

    # 4. Calculate total expected number of values and reshape
    # GrADS binary data is commonly organized as X (longitude) fastest, then Y (latitude),
    # then Z (vertical level), then Variable, then Time (slowest).
    # So the flat file content conceptually follows this order:
    # for t in times:
    #   for z in z_levels:
    #     for var in variables:
    #       for y in latitudes:
    #         for x in longitudes:
    #           read_value()

    # The total number of elements in the binary file expected by the CTL
    total_expected_elements = (metadata['tdef']['count'] *
                               metadata['zdef']['count'] *
                               metadata['num_vars'] * # Each variable block
                               metadata['ydef']['count'] *
                               metadata['xdef']['count'])

    try:
        # Get the actual file size in bytes
        actual_file_size_bytes = os.path.getsize(data_file_path)
        expected_file_size_bytes = total_expected_elements * base_dtype().itemsize

        if actual_file_size_bytes > expected_file_size_bytes:
            print(f"Warning: Data file '{data_file_path}' is larger than expected.")
            print(f"Expected data size: {expected_file_size_bytes} bytes ({total_expected_elements} elements)")
            print(f"Actual file size: {actual_file_size_bytes} bytes ({actual_file_size_bytes / base_dtype().itemsize} elements)")
            print("Proceeding by reading only the expected number of data elements.")
        elif actual_file_size_bytes < expected_file_size_bytes:
            raise ValueError(f"Error: Data file '{data_file_path}' is smaller than expected. "
                             f"Expected {expected_file_size_bytes} bytes but found {actual_file_size_bytes} bytes. Conversion aborted.")

        # Read the binary data, reading only the expected number of elements
        # This will ignore any trailing bytes if the file is larger than expected
        raw_data = np.fromfile(data_file_path, dtype=data_dtype, count=total_expected_elements)
        
        # This check should now always pass if count is used, but keep for robustness if count changes
        if raw_data.size != total_expected_elements:
             # This should ideally not happen if count is specified correctly.
             # It might indicate an underlying issue with np.fromfile or a very strange file.
             raise ValueError(f"Internal error: Read {raw_data.size} elements, but expected {total_expected_elements} after specifying count.")


        # Reshape the raw data.
        # The reshape order is (Time, Z, Variables, Lat, Lon)
        # Using 'F' (Fortran-like) order because X is the fastest varying dimension in GrADS binary.
        data_shape = (
            metadata['tdef']['count'],
            metadata['zdef']['count'],
            metadata['num_vars'], 
            metadata['ydef']['count'],
            metadata['xdef']['count']
        )
        reshaped_data = raw_data.reshape(data_shape, order='F') 

        # 5. Create DataArrays for each variable and populate the Dataset
        data_vars = {}
        for i, var_info in enumerate(metadata['variables']):
            # Slice out the data for each variable from the reshaped array
            # The structure is (time, z, variable_index, lat, lon)
            var_data_array = reshaped_data[:, :, i, :, :]

            # Apply undef value mask
            var_data_array[var_data_array == metadata['undef']] = np.nan

            # Use .squeeze() to remove any singleton dimensions (e.g., Z if zdef 1)
            # This makes the DataArray more natural for 2D or 3D data
            squeezed_data = var_data_array.squeeze()
            
            # Dynamically determine dimensions for the squeezed data
            # This logic is already correct
            dims = []
            if metadata['tdef']['count'] > 1: dims.append('time')
            if metadata['zdef']['count'] > 1: dims.append('z')
            dims.append('lat')
            dims.append('lon')

            # Dynamically create the coords dictionary for each DataArray
            # Only include coordinates for dimensions that are actually present
            current_data_array_coords = {}
            if 'time' in dims:
                current_data_array_coords['time'] = time_coords
            if 'z' in dims:
                current_data_array_coords['z'] = z_coords
            current_data_array_coords['lat'] = y_coords
            current_data_array_coords['lon'] = x_coords

            data_vars[var_info['name']] = xr.DataArray(
                squeezed_data,
                coords=current_data_array_coords, # Use the dynamically created coords
                dims=dims, # Use dynamically determined dimensions
                name=var_info['name'],
                attrs={'long_name': var_info['description'], 'units': 'kg/m^2/s'} # Units hardcoded from your CTL
            )

        # 6. Create the xarray Dataset
        # The main Dataset should still contain all original coordinates as full dimensions,
        # even if individual DataArrays within it have squeezed dimensions.
        ds = xr.Dataset(
            data_vars=data_vars,
            coords={
                'time': time_coords,
                'z': z_coords,
                'lat': y_coords,
                'lon': x_coords
            },
            attrs={'title': metadata.get('title', 'Converted from GrADS binary')}
        )

        # Ensure the output directory exists
        output_dir = os.path.dirname(output_nc_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # 7. Save to NetCDF
        ds.to_netcdf(output_nc_file)
        print(f"Successfully converted '{ctl_file_path}' to '{output_nc_file}'")

    except FileNotFoundError:
        print(f"Error: Data file '{data_file_path}' not found. Please ensure the .dat file exists and the `dset` path in your .ctl is correct relative to the .ctl file.")
    except Exception as e:
        print(f"An unexpected error occurred during conversion: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for detailed debugging

In [35]:
# --- How to use ---
# Define the paths to your .ctl and desired output .nc files
# Ensure 'example_GrADS' directory exists and contains the .ctl and .dat files
# Ensure 'converted_test' directory exists or will be created by the script
ctl_file = "example_GrADS/DecIC_nmme_precip_skill.ctl"
output_netcdf_file = "converted_test/DecIC_nmme_precip_skill.nc"

In [36]:
# Call the conversion function
convert_grads_to_netcdf(ctl_file, output_netcdf_file)

Successfully converted 'example_GrADS/DecIC_nmme_precip_skill.ctl' to 'converted_test/DecIC_nmme_precip_skill.nc'
